In [1]:
from ipywidgets import interact, widgets, Output
from IPython.display import display, clear_output

import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
alto, ancho = 400, 400

# 1- Generación del Phantom

In [3]:
phantom = np.zeros((alto, ancho), dtype=np.uint8)
ellipses = np.array([])

In [4]:
def draw_ellipses (ellipses):
  phantom = np.zeros((ancho, alto))

  for ellipse in ellipses:
    aux = np.zeros_like(phantom)

    if ellipse["I"] > 0:      # Si la elipse es positiva, la ploteo y la sumo
      color = int(ellipse["I"]*255)
      cv2.ellipse(aux, (ellipse["CX"], ellipse["CY"]), (ellipse["X"], ellipse["Y"]), ellipse["A"], 0, 360, color, -1)
      phantom += aux

    elif ellipse["I"] < 0:    # Si la elipse es negativa, la ploteo y la resto
      color = -int(ellipse["I"]*255)
      cv2.ellipse(aux, (ellipse["CX"], ellipse["CY"]), (ellipse["X"], ellipse["Y"]), ellipse["A"], 0, 360, color, -1)
      phantom -= aux

  return np.clip(phantom, 0, 255)/255 # Acoto el phantom entre 0 y 1

def load_ellipse(boton):
    global ellipses, phantom

    aux = {"I": I_box.value,
           "A": A_box.value,
           "X": int((X_box.value)*(ancho//2)),
           "Y": int((Y_box.value)*(alto//2)),
           "CX": int((CX_box.value+1)*ancho//2),
           "CY": int((CY_box.value+1)*alto//2)}

    ellipses = np.append(ellipses, aux)   # Agrego la nueva elipse a la lista
    phantom = draw_ellipses(ellipses)     # Calculo el Phantom
    plot_phantom(phantom_plot, phantom)   # Dibujo el Phantom

def plot_phantom(phantom_plot, phantom):
  with phantom_plot:
    clear_output()
    plt.title('Phantom')
    plt.imshow(phantom, cmap='gray', vmin=0, vmax=1)
    plt.axis('off')
    plt.show()

In [5]:
# Textboxes
I_box = widgets.BoundedFloatText(value=1., min=-1., max=1., step=0.1, description='Intensidad:')
A_box = widgets.IntText(value=0, description='Inclinación:')
X_box = widgets.BoundedFloatText(value=0.25, min=0., max=1., step=0.1, description='Semi-Eje X:')
Y_box = widgets.BoundedFloatText(value=0.25, min=0., max=1., step=0.1, description='Semi-Eje Y:')
CX_box = widgets.BoundedFloatText(value=0., min=-1., max=1., step=0.1, description='Centro X:')
CY_box = widgets.BoundedFloatText(value=0., min=-1., max=1., step=0.1, description='Centro Y:')

# Botón Carga de elipses
load_btn = widgets.Button(description="Cargar")
load_btn.on_click(load_ellipse)

# Plot Phantom
phantom_plot = Output()
plot_phantom(phantom_plot, phantom)

# Mostrar UI
data_box = widgets.VBox([I_box, A_box, X_box, Y_box, CX_box, CY_box, load_btn])
main_box = widgets.HBox([data_box, phantom_plot])
display(main_box)

# 2- Transformada de Radon

In [9]:
from ipywidgets import widgets
from skimage.transform import radon
import IPython.display as ipd

sinogram = None   # Transformada de Radon
theta = []

# Widgets
tita_slider = widgets.FloatRangeSlider(
    min=0,
    max=180,
    step=0.01,
    description='θ [°]:',
)

pasos_num = widgets.IntText(
    value=100,
    description='# pasos:',
    layout=widgets.Layout(width='auto', min_width='150px')
)

calcular_btn = widgets.Button(
    description='Calcular',
    tooltip='Calcular',
)

tita_num = widgets.FloatText(
    value= np.mean(tita_slider.value),
    description='Ver θ [°]:',
    layout=widgets.Layout(width='auto', min_width='150px')
)

ver_btn = widgets.Button(
    description='ver',
    tooltip='Calcular',
)

ver_hBox = widgets.HBox([tita_num, ver_btn])

tita_plot = widgets.Output()
radon_plot = widgets.Output()

plots_hBox = widgets.HBox([radon_plot, tita_plot])

pasos_hbox = widgets.HBox([pasos_num, calcular_btn])

control_vBox = widgets.VBox([tita_slider, pasos_hbox, ver_hBox])

main_vBox = widgets.HBox([control_vBox, plots_hBox])

def calcRadon(*args, **kwargs):
  global theta, sinogram, phantom
  tMin, tMax = tita_slider.value
  theta = np.linspace(tMin, tMax, pasos_num.value, endpoint=False)
  sinogram = radon(phantom, theta=theta)
  with radon_plot:
    ipd.clear_output()
    plt.figure(figsize=(5,5))
    plt.imshow(sinogram, cmap='gray', extent=(tMin, tMax, 0, sinogram.shape[0]), aspect='auto')
    plt.title(r'$Transformada\ de\ Radon$')
    plt.xlabel(r'$Ángulo\ \theta\ [\degree$]')
    plt.ylabel(r'$Pixel$')
    plt.show()


def showRadon(*args, **kwargs):
  global phantom
  tita = tita_num.value
  view = radon(phantom, theta=[tita])[:,0]
  with tita_plot:
    ipd.clear_output()
    plt.figure(figsize=(5,5))
    plt.title(r'Gráfico para $\theta = ' + str(tita) + '\degree$')
    plt.xlabel(r'$Pixel$')
    plt.ylabel(r'$Intensidad$')
    plt.plot(view)
    plt.show()

calcular_btn.on_click(calcRadon)
ver_btn.on_click(showRadon)

# Mostrar la caja de texto y el botón en la interfaz de usuario
ipd.display(main_vBox)

# 3- Obtención del objeto

In [7]:
from skimage.transform import iradon

def plot_inverse(inverse_plot, inverse_mat):
  with inverse_plot:
    clear_output()
    plt.title('Reconstrucción')
    plt.imshow(inverse_mat, cmap='gray', vmin=0, vmax=1)
    plt.axis('off')
    plt.show()

def calc_inverse(button):
  global inverse_mat, sinogram

  inverse_mat=iradon(sinogram, theta=theta, filter_name=selected_filter)
  plot_inverse(inverse_plot, inverse_mat)

def handle_dropdown_change(change):
    selected_filter = change.new

In [8]:
filters = ['ramp', 'shepp-logan', 'cosine', 'hamming', 'hann']
selected_filter = filters[0]
inverse_mat = np.zeros_like(phantom)

# Dropdown tipo de filtro
dropdown = widgets.Dropdown(
    options=filters,
    description='Filtro:'
)

dropdown.observe(handle_dropdown_change, names='value')

# Botón Calculo de inversa
inv_btn = widgets.Button(description="Calcular")
inv_btn.on_click(calc_inverse)

# Plot Inverse
inverse_plot = Output()
plot_inverse(inverse_plot, inverse_mat)

# Mostrar UI
inv_box = widgets.VBox([dropdown, inv_btn])
main_inv = widgets.HBox([inv_box, inverse_plot])
display(main_inv)